In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import EarlyStopping
from tqdm.keras import TqdmCallback
import numpy as np
import matplotlib.pyplot as plt
import pickle

class LstmController:
    
    def __init__(self, s_x, s_y, input_shape, window, layers, drop, units):
        self.temp_history = []
        self.err_history = []
        self.input_shape = input_shape
        self.layers = layers
        self.window = window
        self.drop = drop
        self.units = units
        self.model = self.__create_model()
        self.s_x = s_x
        self.s_y = s_y
        
    @staticmethod
    def load_model(model, params):
        lstm = LstmController(params['Xscale'], params['yscale'], None, params['window'], 0, 0, 0)
        lstm.model = model
        return lstm
        
        
    def save_model(self, model_name, params_name):
        self.model.save(model_name)
        
        model_params = dict()
        model_params['Xscale'] = self.s_x
        model_params['yscale'] = self.s_y
        model_params['window'] = self.window
        pickle.dump(model_params, open(params_name, 'wb'))
        
    
    def train_model(self, Xtrain, ytrain, batch_size):
        es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)
        batch_size = 100
        result = self.model.fit(Xtrain, ytrain, 
                           verbose=0, 
                           validation_split=0.2,
                           callbacks = [es,TqdmCallback(verbose=1)],
                           batch_size=batch_size,
                           epochs=350)
        
        epochs = es.stopped_epoch
        plt.semilogy(result.history['loss'],label='loss')
        plt.semilogy(result.history['val_loss'],label='val_loss')
        plt.legend()
        
    def predict(self, x):
        return self.s_y.inverse_transform(self.model.predict(x))
        
    def __create_model(self):
        if self.layers == 0:
            return None
        
        model = Sequential()

        if self.layers == 1:
            model.add(LSTM(units=self.units, input_shape=(self.input_shape[1],self.input_shape[2])))
            model.add(Dropout(rate=self.drop))
        else:
            model.add(LSTM(units=self.units, return_sequences=True, input_shape=(self.input_shape[1],self.input_shape[2])))
            model.add(Dropout(rate=self.drop))
            for i in range(self.layers-2):
                model.add(LSTM(units=self.units,return_sequences=True))
                model.add(Dropout(rate=self.drop))
            model.add(LSTM(units=self.units))
            model.add(Dropout(rate=self.drop))

        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mean_squared_error')

        return model

    def get_output(self, set_point, process_value, dt) -> float:
        
        error = set_point - process_value

        print(self.temp_history + [set_point])
        print(self.err_history + [error])
        X = np.vstack((self.temp_history + [set_point], self.err_history + [error])).T
        Xs = self.s_x.transform(X)
        Xs = np.reshape(Xs, (1, Xs.shape[0], Xs.shape[1]))

        # Predict Q for controller and unscale
        Q1c_s = self.model.predict(Xs)
        Q1c = self.s_y.inverse_transform(Q1c_s)[0][0]

        # Ensure Q1c is between 0 and 100
        Q1c = np.clip(Q1c,0.0,100.0)
        
        self.temp_history.insert(len(self.temp_history) - 1, self.temp_history.pop(0))
        self.temp_history[-1] = set_point
        self.err_history.insert(len(self.err_history) - 1, self.err_history.pop(0))
        self.err_history[-1] = error

        return Q1c